part1.classification color dataset

In this part, we are doing classification based on the color dataset, using the UNET model. Firstly, we need to download the color dataset from the provided url.

In [0]:
import torch 
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import numpy as np
import torchvision.datasets as datasets
from torch.autograd import Variable
import matplotlib.pyplot as plt
from PIL import Image
import os
import copy
import torch.utils.data as data_utils
import requests, zipfile, io
import matplotlib.cm as cm
from matplotlib.colors import ListedColormap


# Device configuration
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

# Hyper parameters
num_epochs = 40
num_classes = 10
batch_size = 8
learning_rate = 0.0001

# ECE5470 dataset
url = 'https://www.via.cornell.edu/ece5470/Lab9data1.zip'
r=requests.get(url).content

#save data in data dir
z = zipfile.ZipFile(io.BytesIO(r))
# os.mkdir('data1')
z.extractall('./data1')

In [0]:
""" Custom datatset loader
    based on https://github.com/utkuozbulak/pytorch-custom-dataset-examples
"""
import pandas as pd
import imageio

class SimpleDataset():
    def __init__(self, data_path, csv_name, transform = None ):
        """
        Args:
            data_path (string): path to the folder where images and csv files are located
            csv_name (string): name of the csv lablel file
            transform: pytorch transforms for transforms and tensor conversion
        """
        # Set path
        self.data_path = data_path
        
        # Read the csv file
        self.data_info = pd.read_csv(data_path + csv_name, header=0)
       
        # First column contains the image paths
        self.image_arr = np.asarray(self.data_info.iloc[:, 0])
        
        # Second column is the labels
        self.label_arr = np.asarray(self.data_info.iloc[:, 1])
        # Calculate len
        self.data_len = len(self.data_info.index)
        
    def __getitem__(self, index):
        # Get image name from the pandas df
        single_image_name = self.image_arr[index]
        # Open image
        a=single_image_name[:-3] + "png"
        single_image_name=a
        img_as_img = imageio.imread(self.data_path + single_image_name)
        # Open segmented mask
        a = single_image_name[:2] + "seg" + single_image_name[2:]
        imseg_name = a[:-3] + "png"
        imgsg_as_img = imageio.imread(self.data_path + imseg_name)
        
        
        # Get label(class) of the image based on the cropped pandas column
        single_image_label = self.label_arr[index]
        
        return (img_as_img, single_image_label, imgsg_as_img)

    def __len__(self):
        return self.data_len

In [3]:
mydata = SimpleDataset( "./data1/", "labels.csv")

#splitting into images and labels 
X  = []
y  = []
Xs = []
for i in range(len(mydata)):
    X.append(mydata[i][0])
    y.append((mydata[i][1]))
    Xs.append(mydata[i][2])


#converting into numpy arrays to enable easy reshaping and other array operations
    
X  = np.asarray(X)
Xs = np.asarray(Xs)
print("Shape of the input image", X.shape, Xs.shape)
y= np.asarray(y)

X  = np.swapaxes(X,1,3)
X  = np.swapaxes(X,2,3)
print("Shape of the input image", X.shape, Xs.shape,y.shape)

Shape of the input image (10000, 16, 16, 3) (10000, 16, 16)
Shape of the input image (10000, 3, 16, 16) (10000, 16, 16) (10000,)


Then we need to seperate the dataset into three parts- training dataset: 8500 datas; validation dataset: 500 datas; testing dataset: 1000 datas

In [4]:
Xtr=X[:8500,:,:,:]/255
Xstr=Xs[:8500,:,:]/255
ytr=y[:8500]

Xval=X[8500:9000,:,:,:]/255 #need to change validation size for 32x32 & 64x64 images
Xsval=Xs[8500:9000,:,:]/255
yval=y[8500:9000]

Xte=X[-1000:,:,:,:]/255 #need to change test size for 32x32 & 64x64 images
Xste=Xs[-1000:,:,:]/255
yte=y[-1000:]


print(Xtr.shape,Xstr.shape,ytr.shape)
print(Xval.shape,Xsval.shape,yval.shape)
print(Xte.shape,Xste.shape,yte.shape)

(8500, 3, 16, 16) (8500, 16, 16) (8500,)
(500, 3, 16, 16) (500, 16, 16) (500,)
(1000, 3, 16, 16) (1000, 16, 16) (1000,)


Then we need to set the train datset dataloader.

In [5]:
batch_size = 8
'''
inputs and segs are your data. 
'''
targets=ytr

inputs = torch.from_numpy(Xtr).float()
targets = torch.from_numpy(targets).float()
segs    = torch.from_numpy(Xstr).float()

print('Size of inputs: {}'. format(inputs.shape))
print('Size of ground truth: {}'. format(segs.shape))
print(targets.shape)

# Dataloader
trainset = data_utils.TensorDataset(inputs, segs, targets)
train_loader = torch.utils.data.DataLoader(dataset=trainset, batch_size= batch_size, 
                                          shuffle=True,drop_last=True)

Size of inputs: torch.Size([8500, 3, 16, 16])
Size of ground truth: torch.Size([8500, 16, 16])
torch.Size([8500])


Next comes the test dataset dataloader.

In [6]:
targets=yte
inputs = torch.from_numpy(Xte).float()
targets = torch.from_numpy(targets).float()
segs    = torch.from_numpy(Xste).float()

print('Size of inputs: {}'. format(inputs.shape))
print('Size of ground truth: {}'. format(segs.shape))
print(targets.shape)

# Dataloader
testset = data_utils.TensorDataset(inputs, segs,targets)
test_loader = torch.utils.data.DataLoader(testset, batch_size= batch_size, 
                                          shuffle=False, drop_last=True)

Size of inputs: torch.Size([1000, 3, 16, 16])
Size of ground truth: torch.Size([1000, 16, 16])
torch.Size([1000])


Build the Convolutional neural network.

In [0]:
# Convolutional neural network (two convolutional layers)
class ConvNet(nn.Module):
    def __init__(self, num_classes=10):
        super(ConvNet, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(3, 16, kernel_size=5, stride=1, padding=2),
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        self.layer2 = nn.Sequential(
            nn.Conv2d(16, 32, kernel_size=5, stride=1, padding=2),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        self.fc = nn.Linear(2*16*16, num_classes)
        
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.reshape(out.size(0), -1)
        out = self.fc(out)
        return out

model = ConvNet(num_classes).to(device)

In [0]:
# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

Then we begin to train the model using the training dataset, and the training accuracy is 89.5%

In [9]:
# Train the model
total_step = len(train_loader)
resultstr = []
resultsval = []
correct = 0
total = 0
model.train()
for epoch in range(num_epochs):
    for i, (images, segs, targets) in enumerate(train_loader):
        segs = segs.to(device)
        segs = torch.reshape(segs, (batch_size,1,16,16))
        labels = targets.to(device)
        
        # Forward pass
        outputs = model(images.cuda())
        loss = criterion(outputs, labels.to(torch.long))
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # O
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted.to(torch.int) == labels.to(torch.int)).sum().item()

        if (i+1) % 50 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], TrLoss: {:.4f}' 
                   .format(epoch+1,num_epochs,i+1, total_step, loss.data))
    
    resultstr.append(loss.data.cpu().to(torch.float))
acc = correct/total
print('Training accuracy: {} %'.format(100*acc))

Epoch [1/40], Step [50/1062], TrLoss: 2.4452
Epoch [1/40], Step [100/1062], TrLoss: 2.2346
Epoch [1/40], Step [150/1062], TrLoss: 2.1296
Epoch [1/40], Step [200/1062], TrLoss: 2.4211
Epoch [1/40], Step [250/1062], TrLoss: 2.3590
Epoch [1/40], Step [300/1062], TrLoss: 2.1951
Epoch [1/40], Step [350/1062], TrLoss: 1.9597
Epoch [1/40], Step [400/1062], TrLoss: 2.4298
Epoch [1/40], Step [450/1062], TrLoss: 2.2214
Epoch [1/40], Step [500/1062], TrLoss: 2.0130
Epoch [1/40], Step [550/1062], TrLoss: 1.8672
Epoch [1/40], Step [600/1062], TrLoss: 1.8156
Epoch [1/40], Step [650/1062], TrLoss: 2.1528
Epoch [1/40], Step [700/1062], TrLoss: 1.9352
Epoch [1/40], Step [750/1062], TrLoss: 1.9456
Epoch [1/40], Step [800/1062], TrLoss: 2.2027
Epoch [1/40], Step [850/1062], TrLoss: 1.7835
Epoch [1/40], Step [900/1062], TrLoss: 1.8281
Epoch [1/40], Step [950/1062], TrLoss: 1.8375
Epoch [1/40], Step [1000/1062], TrLoss: 1.9021
Epoch [1/40], Step [1050/1062], TrLoss: 1.7821
Epoch [2/40], Step [50/1062], TrL

Test the model, the test accuracy is shown to be 82.7%

In [10]:
# Test the model
model.eval()  # eval mode (batchnorm uses moving mean/variance instead of mini-batch mean/variance)
with torch.no_grad():
    correct = 0
    total = 0
    OP = []
    PR = []
    LB = []
    for i, (images, segs, targets) in enumerate(test_loader):
        segs = segs.to(device)
        segs = torch.reshape(segs, (batch_size,1,16,16))
        labels = targets.to(device)
        LB.append(labels.cpu().numpy())
        outputs=model(images.cuda())
        OP.append(outputs.cpu().numpy())
        _, predicted = torch.max(outputs.data, 1)
        PR.append(predicted.cpu().numpy())
        total += labels.size(0)
        correct += (predicted.to(torch.int) == labels.to(torch.int)).sum().item()

print('Test Accuracy of the model on the 1000 test images: {} %'.format(100 * correct / total))

Test Accuracy of the model on the 1000 test images: 82.7 %


In [11]:
from sklearn.metrics import confusion_matrix
import itertools
import matplotlib.pyplot as plt

pred = np.array(PR)
pred = pred.reshape(1000)
print(np.shape(pred))
label = np.array(LB)
label = label.reshape(1000)
print(np.shape(label))
cfm=confusion_matrix(pred, label)
print(cfm)


(1000,)
(1000,)
[[ 83   1   0   3   1   1   9   0   4   1]
 [  0 100   2   0   1   1   2   2   2   2]
 [  1   5  63   6   1   1   1   4   7   0]
 [  0   0   3 100   1   2   0   0   3   3]
 [  0   1   1   1  83   3   1   1   3   8]
 [  0   0   1   5   0  89   0   0   5   1]
 [  5   1   2   1   1   1  77   0   3   0]
 [  0   1   2   5   0   1   0  84   4   4]
 [  0   1   3   7   3   0   1   1  69   0]
 [  1   0   1   1   6   4   0   7   5  79]]


Then we are going to perform clasification based on the red dataset.

In [0]:
# ECE5470 dataset
url = 'https://www.via.cornell.edu/ece5470/Lab9data1r.zip'
r=requests.get(url).content

#save data in data dir
z = zipfile.ZipFile(io.BytesIO(r))
# os.mkdir('data1')
z.extractall('./data')

In [13]:
mydata = SimpleDataset( "./data/", "labels.csv")

#splitting into images and labels 
X  = []
y  = []
Xs = []
for i in range(len(mydata)):
    X.append(mydata[i][0])
    y.append((mydata[i][1]))
    Xs.append(mydata[i][2])


#converting into numpy arrays to enable easy reshaping and other array operations
    
X  = np.asarray(X)
Xs = np.asarray(Xs)
print("Shape of the input image", X.shape, Xs.shape)
y= np.asarray(y)

X  = np.swapaxes(X,1,3)
X  = np.swapaxes(X,2,3)
print("Shape of the input image", X.shape, Xs.shape,y.shape)

Shape of the input image (10000, 16, 16, 3) (10000, 16, 16)
Shape of the input image (10000, 3, 16, 16) (10000, 16, 16) (10000,)


In [14]:
Xtr=X[:8500,:,:,:]/255
Xstr=Xs[:8500,:,:]/255
ytr=y[:8500]

Xval=X[8500:9000,:,:,:]/255 #need to change validation size for 32x32 & 64x64 images
Xsval=Xs[8500:9000,:,:]/255
yval=y[8500:9000]

Xte=X[-1000:,:,:,:]/255 #need to change test size for 32x32 & 64x64 images
Xste=Xs[-1000:,:,:]/255
yte=y[-1000:]


print(Xtr.shape,Xstr.shape,ytr.shape)
print(Xval.shape,Xsval.shape,yval.shape)
print(Xte.shape,Xste.shape,yte.shape)

(8500, 3, 16, 16) (8500, 16, 16) (8500,)
(500, 3, 16, 16) (500, 16, 16) (500,)
(1000, 3, 16, 16) (1000, 16, 16) (1000,)


In [15]:
batch_size = 8
'''
inputs and segs are your data. 
'''
targets=ytr

inputs = torch.from_numpy(Xtr).float()
targets = torch.from_numpy(targets).float()
segs    = torch.from_numpy(Xstr).float()

print('Size of inputs: {}'. format(inputs.shape))
print('Size of ground truth: {}'. format(segs.shape))
print(targets.shape)

# Dataloader
trainset = data_utils.TensorDataset(inputs, segs, targets)
train_loader = torch.utils.data.DataLoader(dataset=trainset, batch_size= batch_size, 
                                          shuffle=True,drop_last=True)

Size of inputs: torch.Size([8500, 3, 16, 16])
Size of ground truth: torch.Size([8500, 16, 16])
torch.Size([8500])


In [16]:
targets=yte
inputs = torch.from_numpy(Xte).float()
targets = torch.from_numpy(targets).float()
segs    = torch.from_numpy(Xste).float()

print('Size of inputs: {}'. format(inputs.shape))
print('Size of ground truth: {}'. format(segs.shape))
print(targets.shape)

# Dataloader
testset = data_utils.TensorDataset(inputs, segs,targets)
test_loader = torch.utils.data.DataLoader(testset, batch_size= batch_size, 
                                          shuffle=False, drop_last=True)

Size of inputs: torch.Size([1000, 3, 16, 16])
Size of ground truth: torch.Size([1000, 16, 16])
torch.Size([1000])


In [0]:
# Convolutional neural network (two convolutional layers)
class ConvNet(nn.Module):
    def __init__(self, num_classes=10):
        super(ConvNet, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(3, 16, kernel_size=5, stride=1, padding=2),
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        self.layer2 = nn.Sequential(
            nn.Conv2d(16, 32, kernel_size=5, stride=1, padding=2),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        self.fc = nn.Linear(2*16*16, num_classes)
        
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.reshape(out.size(0), -1)
        out = self.fc(out)
        return out

model = ConvNet(num_classes).to(device)

In [0]:
# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

Then train the model. The training accuracy is shown to be 94%

In [19]:
# Train the model
total_step = len(train_loader)
resultstr = []
resultsval = []
correct = 0
total = 0
model.train()
for epoch in range(num_epochs):
    for i, (images, segs, targets) in enumerate(train_loader):
        segs = segs.to(device)
        segs = torch.reshape(segs, (batch_size,1,16,16))
        labels = targets.to(device)
        
        # Forward pass
        outputs = model(images.cuda())
        loss = criterion(outputs, labels.to(torch.long))
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # O
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted.to(torch.int) == labels.to(torch.int)).sum().item()

        if (i+1) % 50 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], TrLoss: {:.4f}' 
                   .format(epoch+1,num_epochs,i+1, total_step, loss.data))
    
    resultstr.append(loss.data.cpu().to(torch.float))
acc = correct/total
print('Training accuracy: {} %'.format(100*acc))

Epoch [1/40], Step [50/1062], TrLoss: 2.1846
Epoch [1/40], Step [100/1062], TrLoss: 2.3537
Epoch [1/40], Step [150/1062], TrLoss: 2.2107
Epoch [1/40], Step [200/1062], TrLoss: 2.2016
Epoch [1/40], Step [250/1062], TrLoss: 2.0200
Epoch [1/40], Step [300/1062], TrLoss: 2.1747
Epoch [1/40], Step [350/1062], TrLoss: 2.0467
Epoch [1/40], Step [400/1062], TrLoss: 1.8984
Epoch [1/40], Step [450/1062], TrLoss: 2.1191
Epoch [1/40], Step [500/1062], TrLoss: 1.6439
Epoch [1/40], Step [550/1062], TrLoss: 1.7665
Epoch [1/40], Step [600/1062], TrLoss: 1.9131
Epoch [1/40], Step [650/1062], TrLoss: 1.3672
Epoch [1/40], Step [700/1062], TrLoss: 1.2580
Epoch [1/40], Step [750/1062], TrLoss: 1.4217
Epoch [1/40], Step [800/1062], TrLoss: 1.5680
Epoch [1/40], Step [850/1062], TrLoss: 1.3829
Epoch [1/40], Step [900/1062], TrLoss: 1.0815
Epoch [1/40], Step [950/1062], TrLoss: 1.1230
Epoch [1/40], Step [1000/1062], TrLoss: 1.0205
Epoch [1/40], Step [1050/1062], TrLoss: 1.6397
Epoch [2/40], Step [50/1062], TrL

The test accuracy is shown as 90.4%, and we also printed the confusion matrix below.

In [20]:
# Test the model
model.eval()  # eval mode (batchnorm uses moving mean/variance instead of mini-batch mean/variance)
with torch.no_grad():
    correct = 0
    total = 0
    PR = []
    LB = []
    for i, (images, segs, targets) in enumerate(test_loader):
        segs = segs.to(device)
        segs = torch.reshape(segs, (batch_size,1,16,16))
        labels = targets.to(device)
        LB.append(labels.cpu().numpy())
        outputs=model(images.cuda())
        OP.append(outputs.cpu().numpy())
        _, predicted = torch.max(outputs.data, 1)
        PR.append(predicted.cpu().numpy())
        total += labels.size(0)
        correct += (predicted.to(torch.int) == labels.to(torch.int)).sum().item()

print('Test Accuracy of the model on the 1000 test images: {} %'.format(100 * correct / total))

Test Accuracy of the model on the 1000 test images: 90.4 %


In [21]:
from sklearn.metrics import confusion_matrix
import itertools
import matplotlib.pyplot as plt

pred = np.array(PR)
pred = pred.reshape(1000)
print(np.shape(pred))
label = np.array(LB)
label = label.reshape(1000)
print(np.shape(label))
cfm=confusion_matrix(pred, label)
print(cfm)

(1000,)
(1000,)
[[ 87   0   1   0   0   1   1   0   0   1]
 [  0 104   3   0   0   0   1   1   2   1]
 [  0   4  67   3   0   0   0   1   1   0]
 [  0   0   3 119   0   2   0   0   8   3]
 [  0   0   1   1  84   0   1   1   1   2]
 [  1   0   0   4   0  95   0   0   3   0]
 [  1   1   0   0   0   2  88   0   1   0]
 [  0   0   2   1   2   2   0  90   0   5]
 [  0   0   0   0   2   0   0   0  85   1]
 [  1   1   1   1   9   1   0   6   4  85]]


Then we use the color dataset to test the model trained by the red dataset. First download the color dataset.

In [0]:

# Device configuration
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

# Hyper parameters
num_epochs = 40
num_classes = 10
batch_size = 8
learning_rate = 0.0001

# ECE5470 dataset
url = 'https://www.via.cornell.edu/ece5470/Lab9data1.zip'
r=requests.get(url).content

#save data in data dir
z = zipfile.ZipFile(io.BytesIO(r))
# os.mkdir('data1')
z.extractall('./data1')

In [23]:
mydata = SimpleDataset( "./data1/", "labels.csv")

#splitting into images and labels 
X  = []
y  = []
Xs = []
for i in range(len(mydata)):
    X.append(mydata[i][0])
    y.append((mydata[i][1]))
    Xs.append(mydata[i][2])


#converting into numpy arrays to enable easy reshaping and other array operations
    
X  = np.asarray(X)
Xs = np.asarray(Xs)
print("Shape of the input image", X.shape, Xs.shape)
y= np.asarray(y)

X  = np.swapaxes(X,1,3)
X  = np.swapaxes(X,2,3)
print("Shape of the input image", X.shape, Xs.shape,y.shape)

Shape of the input image (10000, 16, 16, 3) (10000, 16, 16)
Shape of the input image (10000, 3, 16, 16) (10000, 16, 16) (10000,)


In [24]:
Xtr=X[:8500,:,:,:]/255
Xstr=Xs[:8500,:,:]/255
ytr=y[:8500]

Xval=X[8500:9000,:,:,:]/255 #need to change validation size for 32x32 & 64x64 images
Xsval=Xs[8500:9000,:,:]/255
yval=y[8500:9000]

Xte=X[-1000:,:,:,:]/255 #need to change test size for 32x32 & 64x64 images
Xste=Xs[-1000:,:,:]/255
yte=y[-1000:]


print(Xtr.shape,Xstr.shape,ytr.shape)
print(Xval.shape,Xsval.shape,yval.shape)
print(Xte.shape,Xste.shape,yte.shape)

(8500, 3, 16, 16) (8500, 16, 16) (8500,)
(500, 3, 16, 16) (500, 16, 16) (500,)
(1000, 3, 16, 16) (1000, 16, 16) (1000,)


The color dataset is loaded to be the test dataset

In [25]:
targets=yte
inputs = torch.from_numpy(Xte).float()
targets = torch.from_numpy(targets).float()
segs    = torch.from_numpy(Xste).float()

print('Size of inputs: {}'. format(inputs.shape))
print('Size of ground truth: {}'. format(segs.shape))
print(targets.shape)

# Dataloader
testset = data_utils.TensorDataset(inputs, segs,targets)
test_loader = torch.utils.data.DataLoader(testset, batch_size= batch_size, 
                                          shuffle=False, drop_last=True)

Size of inputs: torch.Size([1000, 3, 16, 16])
Size of ground truth: torch.Size([1000, 16, 16])
torch.Size([1000])


Test the color dataset on the model trained using the red dataset. The accuracy is only 22.9%

In [26]:
# Test the model
model.eval()  # eval mode (batchnorm uses moving mean/variance instead of mini-batch mean/variance)
with torch.no_grad():
    correct = 0
    total = 0
    OP = []
    PR = []
    LB = []
    for i, (images, segs, targets) in enumerate(test_loader):
        segs = segs.to(device)
        segs = torch.reshape(segs, (batch_size,1,16,16))
        labels = targets.to(device)
        LB.append(labels.cpu().numpy())
        outputs=model(images.cuda())
        OP.append(outputs.cpu().numpy())
        _, predicted = torch.max(outputs.data, 1)
        PR.append(predicted.cpu().numpy())
        total += labels.size(0)
        correct += (predicted.to(torch.int) == labels.to(torch.int)).sum().item()

print('Test Accuracy of the model on the 1000 test images: {} %'.format(100 * correct / total))

Test Accuracy of the model on the 1000 test images: 22.9 %


In [27]:
from sklearn.metrics import confusion_matrix
import itertools
import matplotlib.pyplot as plt

pred = np.array(PR)
pred = pred.reshape(1000)
print(np.shape(pred))
label = np.array(LB)
label = label.reshape(1000)
print(np.shape(label))
cfm=confusion_matrix(pred, label)
print(cfm)

(1000,)
(1000,)
[[ 2  0  0  0  0  0  1  0  0  0]
 [51 98 45 69 73 28 61 53 65 64]
 [ 8  6 19 12  4  5 11  3 14  6]
 [ 2  0  0 13  3  3  4  0  5  1]
 [ 0  0  0  0  2  0  1  1  1  1]
 [19  3  9 30 11 63  7 17 16 14]
 [ 0  0  0  1  0  1  5  0  0  0]
 [ 8  3  5  2  2  3  1 25  1 10]
 [ 0  0  0  2  0  0  0  0  0  0]
 [ 0  0  0  0  2  0  0  0  3  2]]
